# Categorical Variables - Variáveis ​​Categóricas

#### Três opções:

1. **Eliminar Variáveis Categóricas**
   - A abordagem mais fácil para lidar com variáveis categóricas é simplesmente removê-las do conjunto de dados. Esta abordagem só funcionará bem se as colunas não contiverem informações úteis.

2. **Codificação Ordinal**
   - A codificação ordinal atribui a cada valor exclusivo um número inteiro diferente.
   - Esta abordagem pressupõe uma ordenação das categorias: "Nunca" (0) < "Raramente" (1) < "Na maioria dos dias" (2) < "Todos os dias" (3).

3. **Codificação Quente**
   - Cria novas colunas indicando a presença (ou ausência) de cada valor possível nos dados originais.


    Exemplo:
    <center>
    <table>
      <tr>
        <th>[COLOR]</th>
        <th>[RED]</th>
        <th>[YELLOW]</th>
        <th>[GREEN]</th>
      </tr>
      <tr>
        <td>[Red]</td>
        <td>[1]</td>
        <td>[0]</td>
        <td>[0]</td>
      </tr>
      <tr>
        <td>[Red]</td>
        <td>[1]</td>
        <td>[0]</td>
        <td>[0]</td>
      </tr>
      <tr>
        <td>[Yellow]</td>
        <td>[0]</td>
        <td>[0]</td>
        <td>[0]</td>
      </tr>
      <tr>
        <td>[Green]</td>
        <td>[0]</td>
        <td>[0]</td>
        <td>[1]</td>
      </tr>
      <tr>
        <td>[Yellow]</td>
        <td>[0]</td>
        <td>[1]</td>
        <td>[0]</td>
      </tr>
    </table>
    <center>


## Class

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Ler dados
data = pd.read_csv('CSV/melb_data.csv')

# Separe o alvo dos preditores 
y = data.Price
X = data.drop(['Price'], axis=1)

# Divide os dados em subconjuntos de treinamento e validação 
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# Elimina colunas com valores ausentes (abordagem mais simples) 
cols_with_missing = [col for col in X_train_full.columns if X_train_full[col].isnull().any()] 
X_train_full.drop(cols_with_missing, axis=1, inplace=True)
X_valid_full.drop(cols_with_missing, axis=1, inplace=True)

# "Cardinalidade" significa o número de valores únicos em uma coluna 
# Selecione colunas categóricas com cardinalidade relativamente baixa (conveniente, mas arbitrária) 
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Selecione colunas numéricas
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Mantém apenas as colunas selecionadas 
my_cols = low_cardinality_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [ ]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,0.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,193.0,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,555.0,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,265.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,673.0,-37.76230,144.8272,4217.0


In [ ]:
# Pega o - dtype - das variaveis.

s = (X_train.dtypes == 'object')
print(s, '\n')

object_cols = list(s[s])
print(object_cols, '\n')


object_cols = list(s[s].index) # Seleciona apenas as que são verdadeiras (True)
print("Categorical variables:")
print(object_cols)

Type              True
Method            True
Regionname        True
Rooms            False
Distance         False
Postcode         False
Bedroom2         False
Bathroom         False
Landsize         False
Lattitude        False
Longtitude       False
Propertycount    False
dtype: bool 

[True, True, True] 

Categorical variables:
['Type', 'Method', 'Regionname']


### score_dataset
- Function to measure quality of each approach 
- Função para medir qualidade de cada abordagem

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

### First - Drop Categorical Variables - Eliminar Variáveis ​​Categóricas

In [ ]:
# object se refere a colunas que contêm dados de tipo "objeto". 
# Colunas de tipo "objeto" geralmente contêm strings, texto ou outros tipos de dados não numéricos.

drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])



print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop categorical variables):
175703.48185157913


### Second - Ordinal Encoding - Codificação Ordinal

O scikit-learn possui uma classe chamada `OrdinalEncoder` que pode ser usada para obter codificações ordinais. Para aplicar o codificador ordinal, fazemos um loop sobre as variáveis categóricas e aplicamos o `OrdinalEncoder` separadamente a cada coluna.


Na célula de código abaixo, para cada coluna, atribuímos aleatoriamente cada valor exclusivo a um número inteiro diferente. Esta é uma abordagem comum que é mais simples do que fornecer rótulos personalizados; entretanto, podemos esperar um aumento adicional no desempenho se fornecermos rótulos mais bem informados para todas as variáveis ​​ordinais. 

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

object_cols = list(s[s].index) 

label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

ordinal_encoder = OrdinalEncoder()
label_X_train[object_cols] = ordinal_encoder.fit_transform(X_train[object_cols])
label_X_valid[object_cols] = ordinal_encoder.transform(X_valid[object_cols])

print("MAE from Approach 2 (Ordinal Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

MAE from Approach 2 (Ordinal Encoding):
165936.40548390493


### Third - One Hot Encoding - Uma Codificação Quente


Usamos a classe `OneHotEncoder` do scikit-learn para obter codificações one-hot. Existem vários parâmetros que podem ser usados para personalizar seu comportamento.

- Configuramos `handle_unknown='ignore'` para evitar erros quando os dados de validação contêm classes que não estão representadas nos dados de treinamento.

- Além disso, definimos `sparse=False` para garantir que as colunas codificadas sejam retornadas como uma matriz numpy (em vez de uma matriz esparsa).


Para usar o codificador, fornecemos apenas as colunas categóricas que desejamos que tenham codificação one-hot. Por exemplo, para codificar os dados de treinamento, fornecemos `X_train[object_cols]`. A lista `object_cols` na célula de código abaixo contém os nomes das colunas com dados categóricos, e, portanto, `X_train[object_cols]` contém todos os dados categóricos no conjunto de treinamento.


Na versão 1.2 do scikit-learn, o parâmetro `sparse` utilizado na classe OneHotEncoder foi renomeado para `sparse_output`. Isso significa que em versões futuras, o uso de sparse em vez de sparse_output gerará um aviso de depreciação.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

object_cols = list(s[s].index) 


# Aplique o codificador one-hot a cada coluna com dados categóricos
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))
#print(OH_cols_train.head())


# definir os indices iguais ao dos originais, pois no OH_cols, ele gera novos index
# isso garante que os DataFrames resultantes estejam alinhados aos originais
# Índice removido da codificação one-hot;   coloque-o de volta 
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index
#print(OH_cols_train.head())


num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)


print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

MAE from Approach 3 (One-Hot Encoding):
166089.4893009678


## Exercise: Categorical Variables

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
X = pd.read_csv('CSV/home-data-for-ml-course/train.csv', index_col='Id') 
X_test = pd.read_csv('CSV/home-data-for-ml-course/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)

# To keep things simple, we'll drop columns with missing values
cols_with_missing = [col for col in X.columns if X[col].isnull().any()] 
X.drop(cols_with_missing, axis=1, inplace=True)
X_test.drop(cols_with_missing, axis=1, inplace=True)

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                                      train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [ ]:
X_train.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
619,20,RL,11694,Pave,Reg,Lvl,AllPub,Inside,Gtl,NridgHt,...,108,0,0,260,0,0,7,2007,New,Partial
871,20,RL,6600,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,...,0,0,0,0,0,0,8,2009,WD,Normal
93,30,RL,13360,Pave,IR1,HLS,AllPub,Inside,Gtl,Crawfor,...,0,44,0,0,0,0,8,2009,WD,Normal
818,20,RL,13265,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,Mitchel,...,59,0,0,0,0,0,7,2008,WD,Normal
303,20,RL,13704,Pave,IR1,Lvl,AllPub,Corner,Gtl,CollgCr,...,81,0,0,0,0,0,1,2006,WD,Normal


#### Compare different models

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [ ]:
s = (X_train.dtypes == 'object')
#print(s)

object_cols = list(s[s].index)
print(object_cols)

['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'KitchenQual', 'Functional', 'PavedDrive', 'SaleType', 'SaleCondition']


### Step 1: Drop columns with categorical data - Eliminar colunas com dados categóricos

In [ ]:
# Preencha as linhas abaixo: elimine colunas nos dados de treinamento e validação

drop_X_train = X_train.select_dtypes(exclude='object')
drop_X_valid = X_valid.select_dtypes(exclude='object')

print('Score:',score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

Score: 17837.82570776256


### Step 2: Ordinal encoding - Eliminar colunas com dados categóricos

In [39]:
# object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]
# Colunas categóricas nos dados de treinamento
object_cols = []
for col in X_train.columns:
    if X_train[col].dtype == "object":
        object_cols.append(col)


# good_label_cols = [col for col in object_cols if 
#                    set(X_valid[col]).issubset(set(X_train[col]))]
# Colunas que podem ser codificadas em ordinal com segurança
good_label_cols = []
for col in object_cols: 
    # verifica se todas as categorias em X_valid para a coluna col estão presentes em X_train
    if set(X_valid[col]).issubset(set(X_train[col])):
        good_label_cols.append(col)


# bad_label_cols = list(set(object_cols)-set(good_label_cols))
# Colunas problemáticas que serão eliminadas do conjunto de dados
bad_label_cols = []
for col in object_cols:
    if col not in good_label_cols:
        bad_label_cols.append(col)


print('Categorical columns that will be ordinal encoded:', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)


Categorical columns that will be ordinal encoded: ['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'BldgType', 'HouseStyle', 'RoofStyle', 'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'KitchenQual', 'PavedDrive', 'SaleType', 'SaleCondition']

Categorical columns that will be dropped from the dataset: ['Condition2', 'RoofMatl', 'Functional']


In [40]:
from sklearn.preprocessing import OrdinalEncoder


ordinal_encoder = OrdinalEncoder()

# Retira as labels ruins
label_X_train = X_train.drop(bad_label_cols, axis=1)
label_X_valid = X_valid.drop(bad_label_cols, axis=1)

label_X_train[good_label_cols] = ordinal_encoder.fit_transform(X_train[good_label_cols])
label_X_valid[good_label_cols] = ordinal_encoder.transform(X_valid[good_label_cols])


print('Score:',score_dataset(label_X_train, label_X_valid, y_train, y_valid))

Score: 17098.01649543379


### Step 3: Investigating cardinality - Investigando Cardinalidade

A Cardinalidade indica quantas ocorrências de uma Entidade participam no mínimo e no máximo do relacionamento

In [54]:
# Obtenha o número de entradas exclusivas em cada coluna com dados categóricos
object_nunique = list(map(lambda col: X_train[col].nunique(), object_cols))
d = dict(zip(object_cols, object_nunique))
print(d)
values = list(d.values())
print(values)

# Imprime o número de entradas únicas por coluna, em ordem crescente
sorted(d.items(), key=lambda x: x[1])

{'MSZoning': 5, 'Street': 2, 'LotShape': 4, 'LandContour': 4, 'Utilities': 2, 'LotConfig': 5, 'LandSlope': 3, 'Neighborhood': 25, 'Condition1': 9, 'Condition2': 6, 'BldgType': 5, 'HouseStyle': 8, 'RoofStyle': 6, 'RoofMatl': 7, 'Exterior1st': 15, 'Exterior2nd': 16, 'ExterQual': 4, 'ExterCond': 5, 'Foundation': 6, 'Heating': 6, 'HeatingQC': 5, 'CentralAir': 2, 'KitchenQual': 4, 'Functional': 6, 'PavedDrive': 3, 'SaleType': 9, 'SaleCondition': 6}
[5, 2, 4, 4, 2, 5, 3, 25, 9, 6, 5, 8, 6, 7, 15, 16, 4, 5, 6, 6, 5, 2, 4, 6, 3, 9, 6]


[('Street', 2),
 ('Utilities', 2),
 ('CentralAir', 2),
 ('LandSlope', 3),
 ('PavedDrive', 3),
 ('LotShape', 4),
 ('LandContour', 4),
 ('ExterQual', 4),
 ('KitchenQual', 4),
 ('MSZoning', 5),
 ('LotConfig', 5),
 ('BldgType', 5),
 ('ExterCond', 5),
 ('HeatingQC', 5),
 ('Condition2', 6),
 ('RoofStyle', 6),
 ('Foundation', 6),
 ('Heating', 6),
 ('Functional', 6),
 ('SaleCondition', 6),
 ('RoofMatl', 7),
 ('HouseStyle', 8),
 ('Condition1', 9),
 ('SaleType', 9),
 ('Exterior1st', 15),
 ('Exterior2nd', 16),
 ('Neighborhood', 25)]

In [63]:
high_cardinality_numcols = len([n for n in values if n > 10])
print(high_cardinality_numcols)

num_cols_neighborhood = (d['Neighborhood'])
print(num_cols_neighborhood)

3
25


Para grandes conjuntos de dados com muitas linhas, a codificação one-hot pode expandir bastante o tamanho do conjunto de dados. Por esse motivo, normalmente codificaremos apenas `colunas one-hot com cardinalidade relativamente baixa`. Então, colunas de alta cardinalidade podem ser eliminadas do conjunto de dados ou podemos usar codificação ordinal.

In [64]:
# Preencha a linha abaixo: Quantas entradas são adicionadas ao conjunto de dados 
# substituindo a coluna por uma codificação one-hot?

OH_entries_added = 1e4*100 - 1e4

OH_entries_added

990000.0